#### Introduction to Delta Lake
#### DS 5110: Big Data Analytics
#### University of Virginia
Last updated: June 11, 2021

#### Sources

Apache Delta  
https://docs.delta.io/latest/delta-intro.html

https://docs.delta.io/latest/quick-start.html#language-python

https://docs.delta.io/latest/delta-utility.html#language-python

Data Lake  
https://aws.amazon.com/big-data/datalakes-and-analytics/what-is-a-data-lake/

Data Lakehouse  
https://databricks.com/blog/2020/01/30/what-is-a-data-lakehouse.html

#### Learning Outcomes

- Understand properties of data lake, data lakehouse
- Understand shortcomings of data lakes, and how data lakehouses address these shortcomings
- Work with Apache Delta lakes to implement their salient features (create, delete, update, conditional update, time travel)
- Understand the benefits of Delta lakes

#### Purpose: Introduce Apache Delta Lake

#### Prerequisites

- *data lake*

Definition: 

A data lake is a centralized repository that allows for storing data of any structure (unstructured/semi-structured/structured) and type (e.g., images, wave files) at any scale.

You can store your data as-is, without having to first structure the data, and run different types of analytics—from dashboards and visualizations to big data processing, real-time analytics, and machine learning to guide better decisions. [source](https://aws.amazon.com/big-data/datalakes-and-analytics/what-is-a-data-lake/)

Shortcomings:

- mention data swamp
- does not support [ACID transactions](https://en.wikipedia.org/wiki/ACID)

- *data lakehouse*

Definition: Combines elements of data lakes and data warehouses for Online Analytics Processing (OLAP) workloads.  
System design provides data management features similar to databases directly on scalable storage used for data lakes (e.g., AWS S3).

Benefits: 

- supports ACID transactions
- supports diverse data types
- open source storage format
- provides transactional guarantees
- supports diverse workloads
- enables schema enforcement and evolution
- supports upserts and deletes

**Delta Lake**  
Open source project, enables building Lakehouse architecture on top of data lakes. 

Includes the data lakehouse properties above, plus:
- unifies streaming & batch processing on top of existing data lakes,   
  like S3, Azure Data Lake Storage (ADLS), Google Cloud Storage, HDFS
- enables time travel

import Delta Lake

In [0]:
from delta import *
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

### Reading and Writing with Delta Table

Create a **Delta table** by creating some data and save in "delta" format at path: */tmp/delta-table*

In [0]:
data = spark.range(0, 10)
data.write.format("delta").save("/tmp/delta-table")

In [0]:
# delete the data (so we can show we can read from Delta table later)
del data

Read the data from the Delta table

In [0]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
 id|
+---+
113|
 14|
 18|
119|
 12|
 16|
111|
115|
117|
 10|
+---+

Note from **Details** that 8 files are created (which matches number of cores)

In [0]:
# show the first few rows
df.head(3)

Out[22]: [Row(id=0), Row(id=1), Row(id=2)]

Ok so we can write to Delta lake and read from Delta lake. Next, let's update the data.  
`mode("overwrite")` will overwrite the saved data.

In [0]:
data = spark.range(10, 20)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

Reading in the updated data

In [0]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
 id|
+---+
 13|
 14|
 18|
 19|
 11|
 12|
 15|
 16|
 17|
 10|
+---+

### Using **Time Travel** to read old versions of the data

Time travel allows for querying **previous snapshots** of a Delta table.  
This is implemented with *versionAsOf* option.

In [0]:
df0 = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df0.show()

+---+
 id|
+---+
 3|
 4|
 8|
 9|
 0|
 1|
 2|
 5|
 6|
 7|
+---+

**TRY FOR YOURSELF**  
Update the data a second time, storing this third version of the data in */tmp/delta-table2*  
Then use time travel to read in the second version of the data. Convince yourself you did this properly.

### Conditional Updating without Overwrite  

Delta lake supports the following operations:

- conditional update
- delete
- merge (upsert) data into tables; upsert will insert new rows, and update pre-existing rows

#### Conditional Update: update the odd values by adding 100

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

# set the path
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# update logic for the delta
deltaTable.update(
  condition = expr("id % 2 == 1"),
  set = { "id": expr("id + 100") })

deltaTable.toDF().show()

+---+
 id|
+---+
113|
 14|
 18|
119|
 12|
 16|
111|
115|
117|
 10|
+---+

### Get the History of the Table

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

fullHistoryDF = deltaTable.history()    # get the full history of the table

lastOperationDF = deltaTable.history(1) # get the last operation

Look at full history

In [0]:
fullHistoryDF.show()

+-------+-------------------+----------------+------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
version| timestamp| userId| userName|operation| operationParameters| job| notebook| clusterId|readVersion| isolationLevel|isBlindAppend| operationMetrics|userMetadata|
+-------+-------------------+----------------+------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
 2|2021-06-07 18:15:47|4488970776501633|apt4c@virginia.edu| UPDATE|[predicate -> ((i...|null|[3324046535357069]|0607-154549-robin381| 1|WriteSerializable| false|[numRemovedFiles ...| null|
 1|2021-06-07 18:08:34|4488970776501633|apt4c@virginia.edu| WRITE|[mode -> Overwrit...|null|[3324046535357069]|0607-154549-robin381| 0|WriteSerializable| false|[numFiles -> 8, n...| null|
 0|2021-06-07 18:08:12|4488970776501633|apt4c@virginia.edu| WRITE|[mode -> ErrorIfE...|null|[3324046535357069]|0607-154549-robin381| null|WriteSerializable| true|[numFiles -> 8, n...| null|
+-------+-------------------+----------------+------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+

Look at operations in full history

In [0]:
fullHistoryDF.select('version','operation','operationMetrics').show(truncate=False)

+-------+---------+-----------------------------------------------------------------------------------+
version|operation|operationMetrics |
+-------+---------+-----------------------------------------------------------------------------------+
2 |UPDATE |[numRemovedFiles -> 5, numAddedFiles -> 5, numUpdatedRows -> 5, numCopiedRows -> 2]|
1 |WRITE |[numFiles -> 8, numOutputBytes -> 3863, numOutputRows -> 10] |
0 |WRITE |[numFiles -> 8, numOutputBytes -> 3864, numOutputRows -> 10] |
+-------+---------+-----------------------------------------------------------------------------------+

Show Delta table details

Note: we can write SQL in a cell by starting with magic command `%sql`  
The supported magic commands are: `%python`, `%r`, `%scala`, and `%sql`

In [0]:
%sql

DESCRIBE DETAIL "/tmp/delta-table"

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,c4832c8c-5bc5-46b7-95a7-d0e185a600e3,null,null,dbfs:/tmp/delta-table,2021-06-07T18:08:09.838+0000,2021-06-07T18:15:47.000+0000,List(),8,3863,Map(),1,2


**Drop the Delta table**

In [0]:
dbutils.fs.rm('/tmp/delta-table',recurse=True)

Out[17]: True